# Feature Engineering

Objetivo: crear features simples, interpretables y reconstruibles desde el API.
Regla: no crear features no disponibles en inferencia.
Nota: variables de clima se obtienen via API externa usando lat/lon y fecha_partida.


## 1. Cargar flights_model_ready.parquet


In [1]:
import pandas as pd
from pathlib import Path

input_path = Path('..') / '02_LIMPIEZA_Y_DATASET_MODEL_READY' / 'outputs' / 'flights_model_ready.parquet'
df = pd.read_parquet(input_path)
df.shape


(35669175, 15)

## 2. Crear features desde fecha_partida
- hora
- dia de la semana
- mes


In [2]:
df['dep_hour'] = (df['sched_minute_of_day'] // 60).astype('int8')
df['dep_dow'] = (df['FL_DATE'].dt.dayofweek + 1).astype('int8')
df['dep_month'] = df['FL_DATE'].dt.month.astype('int8')


## 3. Preparar variables categoricas


In [3]:
cat_cols = ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')


## 4. Verificar compatibilidad con el JSON del API
Se valida que las variables requeridas existan o puedan derivarse.


In [4]:
required_cols = ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'FL_DATE', 'sched_minute_of_day', 'DISTANCE', 'TEMP', 'WIND_SPD', 'PRECIP_1H', 'CLIMATE_SEVERITY_IDX']
missing = [c for c in required_cols if c not in df.columns]
missing


[]

## 5. Seleccionar features finales y renombrar a contrato API


In [5]:
df['distancia_km'] = (df['DISTANCE'] * 1.60934).astype('float32')
features = df[[
    'OP_UNIQUE_CARRIER',
    'ORIGIN',
    'DEST',
    'distancia_km',
    'dep_hour',
    'dep_dow',
    'dep_month',
    'TEMP',
    'WIND_SPD',
    'PRECIP_1H',
    'CLIMATE_SEVERITY_IDX',
    'DEP_DEL15',
]]
features = features.rename(columns={
    'OP_UNIQUE_CARRIER': 'aerolinea',
    'ORIGIN': 'origen',
    'DEST': 'destino',
    'TEMP': 'temp',
    'WIND_SPD': 'wind_spd',
    'PRECIP_1H': 'precip_1h',
    'CLIMATE_SEVERITY_IDX': 'climate_severity_idx',
})
features.shape


(35669175, 12)

## 6. Guardar dataset final


In [6]:
output_path = Path('outputs') / 'flights_features.parquet'
features.to_parquet(output_path, index=False)
output_path


WindowsPath('outputs/flights_features.parquet')